In [ ]:
# %load annotatorcode.py
# bioportal
import os
import json
import requests
import urllib
from urllib import parse, request

import nltk
from nltk.corpus import stopwords

REST_URL = "http://data.bioontology.org"
API_KEY = '3b00793b-f3cc-489c-9d6e-7a888f4b656e'
PATH_TO_PAPERS = 'kaggle/'
URL_FSERVER = 'http://caribe.udea.edu.co/pmc_murcia/'
ontoltogies = 'MESH,PREMEDONTO,SNOMEDCT,COVID19'

def parse_directory(path=PATH_TO_PAPERS):
    '''For parse and join paths in PATH_TO_PAPERS.
       Output:path_files: list with all paths to record 
    '''
    global path_files
    path_files = []

    for root, dirs, files in os.walk(path):
        for file in files:
            path_files.append(os.path.join(root, file))

    return path_files

def load_json_record(path_to_record):
    ''' Load entire json record for a paper.
        Input: path to record.
        Output: json_record.
    '''
    global json_record
    with open(path_to_record, 'r') as json_handler:
        json_record = json.load(json_handler)
    return json_record

def make_url_query(text_to_annotate):
    global URL 
    URL = REST_URL + "/annotator/?text=%s"%(parse.quote(text_to_annotate))
    return URL

def get_json(url):
    opener = urllib.request.build_opener()
    opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
    return json.loads(opener.open(url).read())

def call_endpoint_annotator(URL, PARAMS):
    r = requests.get(url = URL, params = PARAMS)
    global json_api_results
    json_api_results = r.json()
    return json_api_results

def store_annotations(annotations,get_class=True):
    anns = []
    for result in annotations:
        ann = {}
        class_details = result["annotatedClass"]
        if get_class:
            try:
                class_details = get_json(result["annotatedClass"]["links"]["self"])
            except urllib.error.HTTPError:
                print(f"Error retrieving {result['annotatedClass']['@id']}")
                continue
        ann['id'] = class_details["@id"]
        ann['preflabel'] = class_details["prefLabel"]
        ann['ontology'] = class_details["links"]["ontology"]
        ann['annotation_details'] =  []
        for annotation in result['annotations']:
            ann['annotation_details'].append(
                {
                    'from': str(annotation["from"]),
                    'to': str(annotation["to"]),
                    'match_type': annotation["matchType"],
                    'text': annotation['text']
                })
        if result["hierarchy"]:
            ann['hierarchy'] = []
            for annotation in result["hierarchy"]:
                try:
                    class_details = get_json(annotation["annotatedClass"]["links"]["self"])
                except urllib.error.HTTPError:
                    print(f"Error retrieving {annotation['annotatedClass']['@id']}")
                    continue
                ann['hierarchy'].append(
                    {
                     'id':class_details["@id"],
                     'preflabel':class_details["prefLabel"],
                     'ontology':class_details["links"]["ontology"],
                     'distance_from_original_class':str(annotation["distance"])
                    })
        anns.append(ann)
    return anns

def get_annotations(text_to_annotate,ontologies,expand_class_hierarchy='false',class_hierarchy_max_level=0):
    '''get annotations from bioPortal for one fragment of text'''
    PARAMS = {'apikey':API_KEY,'ontologies':ontologies,
              'expand_class_hierarchy':expand_class_hierarchy,
              'class_hierarchy_max_level':0,
              'longest_only':'true'
             }
    make_url_query(text_to_annotate)
    call_endpoint_annotator(URL,PARAMS)
    return store_annotations(json_api_results)

def count_annotations(ann):
    count = {}
    for i in ann:
        onto = i['ontology'].split('/')[-1]
        key = onto + "_Matched_Term"
        if onto not in count.keys():
            count[onto] = 1
            count[key] = [i['preflabel']]
        else:
            count[onto] += 1
            count[key].append(i['preflabel'])
    return count

In [ ]:
ontologies = "PREMEDONTO,MESH,COVID19,SNOMEDCT" # ontoportal id ontology.

In [ ]:
parse_directory(path='article_records/')

In [ ]:
pmc_art = load_json_record(path_files[1])

In [ ]:
pmc_art['body_text']

In [ ]:
# falto nombre clase.

In [ ]:
# json article annotations

{
    'id':str(id_db),
    'annotations':{
        'section_x':[],
        'section_y':[],
        ...
        'section_z':[]
    }
    count_annotations:[
        
    ]
}

In [ ]:
class articleProcess:
    def __init__(self,record):
        
        self.record = record
        
        self.pmc_art_ann = {
                            'id':record['paper_id'],
                            'annotations':{},
                            'annotations_count':[],        # conteo de clases por ontologia
                            'annotations_count_resume':{}, # total conteo de clases por ontologia y lista de clases
                            'token_resumen': {}            # 
                            }
        
    def AnnotateRecord(self):
        
        for chunk in self.record['body_text']:
            # get annotation for chunk
            ann = get_annotations(chunk['text'],ontologies)
            print(chunk['text']+'\n')
            # store annotation
            section = chunk['section'].split(':::')[-1]
            if section not in self.pmc_art_ann['annotations'].keys():
                self.pmc_art_ann['annotations'][section] = ann
            else:
                self.pmc_art_ann['annotations'][section].append(ann)
            # count annotations
            ann_count=count_annotations(ann)
            #store annotation count
            self.pmc_art_ann['annotations_count'].append(ann_count)
            
    def Resumen(self):
        # total annotations count
        word_resume = []
        for onto in ontologies.split(','):
            self.pmc_art_ann['annotations_count_resume'][onto]=0
            self.pmc_art_ann['annotations_count_resume'][onto+'_Matched_Term'] = []

        for onto in ontologies.split(','): 
            for i in self.pmc_art_ann['annotations_count']:
                if onto in i.keys():
                    self.pmc_art_ann['annotations_count_resume'][onto] += i[onto]
                    self.pmc_art_ann['annotations_count_resume'][onto + '_Matched_Term'].extend(i[onto+'_Matched_Term'])
                    word_resume.extend(i[onto+'_Matched_Term'])
        # word count
        word_count = {}
        for i in word_resume:
            if i not in word_count.keys():
                word_count[i] = 1
            else:
                word_count[i]+=1

        stops = stopwords.words('english')
        tokens = ''

        for i in ins.record['body_text']:
            tokens=tokens+''+i['text']
            token = [t for t in tokens.split(' ') if t not in stops]

        self.pmc_art_ann['token_resumen']['article_tokens'] = list(set(token))
        self.pmc_art_ann['token_resumen']['count_matches'] = word_count
        self.pmc_art_ann['token_resumen']['all_matches'] = word_resume
        self.pmc_art_ann['token_resumen']['unique_matches'] =  list(set(word_resume))
        self.pmc_art_ann['token_resumen']['porcentage_matched'] = len(self.pmc_art_ann['token_resumen']['unique_matches'])/len(self.pmc_art_ann['token_resumen']['article_tokens'])
        

In [ ]:
ins = articleProcess(pmc_art) # instancia de clase con articulo pmc_murcia

In [ ]:
ins.AnnotateRecord() # generacion de anotaciones

In [ ]:
ins.pmc_art_ann

In [ ]:
ins.Resumen() # resumen de anotaciones y conteo de tokens

In [ ]:
ins.pmc_art_ann.keys() # vista general del json utilizando 4 ontologias

In [ ]:
ins.pmc_art_ann['annotations_count_resume'].keys() # resumen de las anotaciones: conteo por ontologia y clases

In [ ]:
ins.pmc_art_ann['annotations_count_resume']

In [ ]:
ins.pmc_art_ann['token_resumen'].keys() # resumen de tokens

In [ ]:
ins.pmc_art_ann['token_resumen']

In [ ]:
ins.pmc_art_ann['token_resumen']['count_matches']

In [ ]:
# porcentaje por ontologia

############################ 

In [ ]:
file = ins.pmc_art_ann

In [ ]:
with open('pmc_art_0.json','w') as fileH:
    fileH.write(json.dumps(file))

In [ ]:
class handleDB:
    
    def __init__(self):
        pass
    
    # coneccion
    
    # lectura datos
    
    def read_pmc_db(self, path):
        pass

In [ ]:
with open('index.html.tmp','r') as htmlFile:
    html = htmlFile.read()

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html)

In [ ]:
pmcs = [tag['href'] for tag in soup.findAll('a', href=True) if 'PMC' in tag['href']]

In [ ]:
wget.download('http://caribe.udea.edu.co/pmc_murcia/')

In [ ]:
len(pmcs)

In [ ]:
import wget

In [ ]:
help(wget.download)

In [ ]:
def ReadFileServer():
    
    wget.download(URL_FSERVER, out='./article_records/Findex.html')
    
    with open('./article_records/Findex.html','r') as htmlFile:
        html = htmlFile.read()
        
    soup = BeautifulSoup(html)
    
    pmcs = [tag['href'] for tag in soup.findAll('a', href=True) if 'PMC' in tag['href']]
    
    return pmcs

In [ ]:
!wget url ...

In [ ]:
records = ReadFileServer()

In [ ]:
wget.download(URL_FSERVER+records[0],out='./article_records/'+records[0])

In [ ]:
def DownLoadRecords(): 
    records = ReadFileServer()
    print('Server conted read; Num of files:',len(records))
    for r in records:
        wget.download(URL_FSERVER+r,out='./article_records/'+r)

In [ ]:
DownLoadRecords()

In [ ]:
# make class for handle general db
# METHODS
# create db
# store things in db
# load db in memory
# write db in disk

# make class for handle bioportal api service
# METHODS
# handle web servise
# handle virtual machine service
# call annotator

# proccess json
# read file
# map keys
# iterate over keys
# call api with section content
# store annotations for section
# count annotations for each annotated section
# store count
# store annotations and count annotation for file in db

In [ ]:
# data management plan
# -> intelectual property right management
#    -> identification of data
#    -> funder requeriments
#    -> authority
#    -> composite
#    -> sharing requeriments
#        ->permisions
#        conditions
#        restrictions
#        select and grouping
#    -> identifiying license more suitable
#    -> depositing data in data verse
#    -> setting up permissions in dataverse
#       -> diligence metadata fields related to license
#       ->

In [ ]:
###

In [ ]:
'''
OP0:
archivo_anotaciones -> conteo_anotaciones -> datos_para_filtros
OP1: 
db_postgres -> archivo_anotaciones -> conteo -> datos_para_filtros
'''

In [ ]:
'''
Estructura de datos a nivel de articulo:

ann={
    'id':'int',
    'terms':[{
                    'term':'str',
                    'abs_count':'int',
                    'features':[
                                    {
                                        'section':'str',
                                        'ontology':'str',
                                        'class':'str'
                                    },...,
                                    N
                                ]
                },...,

            N],
            
    'ontos':[{
                    'ontology':'str',
                    'abs_count':'int',
                    'features':[
                                    {
                                        'section':'str',
                                        'class':'str'
                                    }
                               ]
                },...,
            N]}   
        
# Estructura de datos a nivel de base de datos

db = [ann1,...,annN]
'''
'''
    articulo
    seccion
    ontologia
    terminos
'''

In [1]:
import json
import pandas as pd
import sys

# load data
#df = pd.read_csv('export (14).csv')
#dff = df.to_json()
#data = json.loads(dff)

In [34]:
# explore data

In [35]:
data.keys()

dict_keys(['title_section', 'ID', 'part', 'concept', 'term'])

In [36]:
l = list(data['ID'].values())

In [37]:
len(l) # 100 rows

1001666

In [38]:
nones = [x for x in l if x == None ]

In [39]:
len(nones) # rows id nones

31988

In [59]:
# get ids titles
ids = list(set(list(data['ID'].values()))) # whitout nones

# ART INDEX
#{
#   'id':['fst_index','last_index'],
#   'id':['fst_index','last_index']
#}
art_indx = {} 

for i in ids:
    art_indx[str(i)] = []

# get range of index of each title
for k in data['ID'].keys():
    _id = str(data['ID'][k])  
    if len(art_indx[_id]) == 0:
        art_indx[_id].append(k)
    elif len(art_indx[_id]) == 1:
        art_indx[_id].append(k)
    elif len(art_indx[_id]) == 2:
        art_indx[_id][-1] = k

In [61]:
len(list(art_indx.keys()))

14613

In [2]:
path_to_csv_file = 'export (14).csv'
def read_data(path_to_csv_file):
    df = pd.read_csv(path_to_csv_file)

    dff = df.to_json()

    data = json.loads(dff)
    
    return data

In [3]:
def get_db_index(data):
    # get ids titles
    ids = list(set(list(data['ID'].values()))) # whitout nones

    # ART INDEX
    #{
    #   'id':['fst_index','last_index'],
    #   'id':['fst_index','last_index']
    #}
    art_indx = {} 

    for i in ids:
        art_indx[str(i)] = []

    # get range of index of each title
    for k in data['ID'].keys():
        _id = str(data['ID'][k])  
        if len(art_indx[_id]) == 0:
            art_indx[_id].append(k)
        elif len(art_indx[_id]) == 1:
            art_indx[_id].append(k)
        elif len(art_indx[_id]) == 2:
            art_indx[_id][-1] = k
    return art_indx

In [4]:
def generate_json_ann_counts_db(art_index,data):
    db = []
    for i in art_index.keys():
        print(i)
        _id = i
        ann = {'id':_id,
          'terms':[],
          'onts':[]
          }

        _range = art_index[_id]

        terms = []

        all_all = []

        ontos = []

        for i in range(int(_range[0]),int(_range[-1])+1):
            print(i)

            title_section = data['title_section'][str(i)]

            try:

                ontology = data['part'][str(i)].split('/')[-1]

            except AttributeError:

                ontology = 'None'

            clase = data['concept'][str(i)]

            termino = data['term'][str(i)]

            terms.append(termino)

            ontos.append(ontology)

            _all = [termino, title_section, ontology, clase]

            all_all.append(_all)

        # terms
        unique_terms = list(set(terms))
        terms = []
        for i in unique_terms:
            terms.append({
                'term':i,
                'abs_count':0,
                'features':[]

            })

        for i in terms:
            for j in all_all:
                if i['term'] == j[0]:
                    i['abs_count'] += 1
                    i['features'].append(
                    {
                        'section':j[1],
                        'ontology':j[2],
                        'class':j[3]
                    })           

        # ontos
        unique_ontos = list(set(ontos))
        ontos = []
        for i in unique_ontos:
            ontos.append({
                'ontology':i,
                'abs_count':0,
                'features':[]

            })

        for i in ontos:
            for j in all_all:
                if i['ontology'] == j[2]:
                    i['abs_count'] += 1
                    i['features'].append(
                    {
                        'section':j[1],
                        'class':j[3]
                    })  

        # Store terms count and ontologies count
        ann['terms'] = terms
        ann['onts'] = ontos
        db.append(ann)
        
        return db

In [5]:
data = read_data('export (14).csv')
    
art_index = get_db_index(data)
  

In [6]:
art_index

{'33095680.0': ['185192', '185337'],
 '15761412.0': ['628837', '628888'],
 '33161221.0': ['807463', '807538'],
 '32210950.0': ['682263', '682461'],
 '33456134.0': ['862417', '862472'],
 '32768012.0': ['206279', '206359'],
 '25559054.0': ['444695', '444750'],
 '33259536.0': ['293147', '293166'],
 '32309267.0': ['895626', '895650'],
 '32047128.0': ['254269', '254507'],
 '16580632.0': ['787590', '787641'],
 '32047134.0': ['92463', '92663'],
 '25559071.0': ['79825', '79917'],
 '31752224.0': ['727352', '727410'],
 '32833573.0': ['164726', '164771'],
 '33062953.0': ['938562', '938676'],
 '20217898.0': ['452495', '452652'],
 '16154667.0': ['936943', '936998'],
 '33128497.0': ['336809', '336858'],
 '33390643.0': ['595088', '595117'],
 '32800824.0': ['75872', '75902'],
 '33587262.0': ['708903', '708953'],
 '33292351.0': ['284616', '284645'],
 '23953475.0': ['862349', '862395'],
 '28803142.0': ['483289', '483467'],
 '33521739.0': ['674264', '674286'],
 '33292368.0': ['461460', '461493'],
 '33456

In [7]:

db = generate_json_ann_counts_db(art_index, data)
    

33095680.0
185192
185193
185194
185195
185196
185197
185198
185199
185200
185201
185202
185203
185204
185205
185206
185207
185208
185209
185210
185211
185212
185213
185214
185215
185216
185217
185218
185219
185220
185221
185222
185223
185224
185225
185226
185227
185228
185229
185230
185231
185232
185233
185234
185235
185236
185237
185238
185239
185240
185241
185242
185243
185244
185245
185246
185247
185248
185249
185250
185251
185252
185253
185254
185255
185256
185257
185258
185259
185260
185261
185262
185263
185264
185265
185266
185267
185268
185269
185270
185271
185272
185273
185274
185275
185276
185277
185278
185279
185280
185281
185282
185283
185284
185285
185286
185287
185288
185289
185290
185291
185292
185293
185294
185295
185296
185297
185298
185299
185300
185301
185302
185303
185304
185305
185306
185307
185308
185309
185310
185311
185312
185313
185314
185315
185316
185317
185318
185319
185320
185321
185322
185323
185324
185325
185326
185327
185328
185329
185330
185331
185332
18

In [105]:
len(db)

1

In [86]:
art_index

{'33095680.0': ['185192', '185337'],
 '15761412.0': ['628837', '628888'],
 '33161221.0': ['807463', '807538'],
 '32210950.0': ['682263', '682461'],
 '33456134.0': ['862417', '862472'],
 '32768012.0': ['206279', '206359'],
 '25559054.0': ['444695', '444750'],
 '33259536.0': ['293147', '293166'],
 '32309267.0': ['895626', '895650'],
 '32047128.0': ['254269', '254507'],
 '16580632.0': ['787590', '787641'],
 '32047134.0': ['92463', '92663'],
 '25559071.0': ['79825', '79917'],
 '31752224.0': ['727352', '727410'],
 '32833573.0': ['164726', '164771'],
 '33062953.0': ['938562', '938676'],
 '20217898.0': ['452495', '452652'],
 '16154667.0': ['936943', '936998'],
 '33128497.0': ['336809', '336858'],
 '33390643.0': ['595088', '595117'],
 '32800824.0': ['75872', '75902'],
 '33587262.0': ['708903', '708953'],
 '33292351.0': ['284616', '284645'],
 '23953475.0': ['862349', '862395'],
 '28803142.0': ['483289', '483467'],
 '33521739.0': ['674264', '674286'],
 '33292368.0': ['461460', '461493'],
 '33456

In [83]:
len(db)

1

In [48]:
len(db)

1

In [24]:
db[0].keys()

dict_keys(['id', 'terms', 'onts'])

In [25]:
db[0]['terms']

[{'term': 'RT-PCR',
  'abs_count': 3,
  'features': [{'section': 'WHO Case Definitions of COVID-19 and RT-PCR Tests',
    'ontology': 'ochv',
    'class': 'http://sbmi.uth.tmc.edu/ontology/ochv#41989'},
   {'section': 'WHO Case Definitions of COVID-19 and RT-PCR Tests',
    'ontology': 'id',
    'class': 'http://purl.jp/bio/4/id/200906089061584649'},
   {'section': 'WHO Case Definitions of COVID-19 and RT-PCR Tests',
    'ontology': 'ochv',
    'class': 'http://sbmi.uth.tmc.edu/ontology/ochv#C0599161'}]},
 {'term': 'CHEST CT',
  'abs_count': 18,
  'features': [{'section': 'Role of Chest CT in Diagnostic Decision Making',
    'ontology': 'id',
    'class': 'http://purl.jp/bio/4/id/200906069720605992'},
   {'section': 'Chest CT Appearance of COVID-19',
    'ontology': 'id',
    'class': 'http://purl.jp/bio/4/id/200906069720605992'},
   {'section': 'Reporting and Communicating Chest CT Findings',
    'ontology': 'id',
    'class': 'http://purl.jp/bio/4/id/200906069720605992'},
   {'sectio

In [16]:
# store
with open('db_ann_json.json','w') as jh:
    jh.write(json.dumps(db))

In [27]:
with open('db_ann_json.json','r') as jh:
    data = json.loads(jh.read())

In [28]:
len(data)

1

In [ ]:
if __name__ == "__main__":
    
    path_to_file = sys.argv[1]
    
    data = read_data(path_to_file)
    
    art_index = get_db_index(data)
    
    db = generate_json_ann_counts_db(art_index, data)
    
    # store
    with open('db_ann_json.json','w') as jh:
        jh.write(json.dumps(db))